In [1]:
#drive.mount('/content/drive/')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
import imageio
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression 
from sklearn import datasets
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms

In [2]:
math = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/ECGR 4105 - Intro to Machine Learning/ECGR 4105 - Final Project/Portuguese.csv")) 
math.head() 

FileNotFoundError: ignored

In [ ]:
m = len(math) 
m 

In [ ]:
math.shape 

In [ ]:
math.describe()

In [ ]:
math.describe().T

In [ ]:
math.nunique()

In [ ]:
math.info() 

In [ ]:
math.isnull().sum()

In [ ]:
num_cols=math.select_dtypes(include=['float64', 'int64'])
num_cols.head()

In [ ]:
cat_cols=math.select_dtypes(include=['object'])
cat_cols.head()

In [ ]:
math1=math.copy()

In [ ]:
#need to clean up dataset
def clean_data(df):
    temp = df
    x = df.columns.tolist()   #turns the column titles into a list
    types = df.dtypes           #returns the type of each column
    for i in range(len(x)):
        if types[i] == 'str' or types[i] == 'object':
            temp[x[i]] = df[x[i]].astype('category').cat.codes
        else: 
            temp[x[i]] = df[x[i]]
    return temp

In [ ]:
math = clean_data(math)
df = math
df

In [ ]:
# You can see that your dataset has many columns with values as 'Yes' or 'No'. 
# But in order to fit a regression line, we would need numerical values and not string.
# List of variables to map 
 
varlist =  ['age', 'sex', 'studytime', 'famsup', 'paid', 'activities', 'higher', 'internet', 'Dalc', 'Walc', 'absences', 'G3'] 
 
# Defining the map function 
#def binary_map(x): 
#    return x.map({'yes': 1, "no": 0, "M": 0, "F": 1}) 
 
# Applying the function to the math list 
math = math[varlist]
 
# Check the math dataframe now 
math.head() 

In [ ]:
#Splitting the Data into Training and Testing Sets 
from sklearn.model_selection import train_test_split 
 
# We specify this so that the train and test data set always have the same rows, respec
np.random.seed(0) 
df_train, df_test = train_test_split(math, train_size = 0.8, test_size = 0.2, random_state=42)
df_train.shape

In [ ]:
df_test.shape 

In [ ]:
num_vars = ['age', 'sex', 'studytime', 'famsup', 'paid', 'activities', 'higher', 'internet', 'Dalc', 'Walc', 'absences', 'G3'] 
df_Newtrain = df_train[num_vars] 
df_Newtest = df_test[num_vars] 
df_Newtrain.head() 

In [ ]:
df_Newtrain.shape 

In [ ]:
# Here we can see that except for area, all the columns have small integer values.  
#So it is extremely important to rescale the variables so that they have a comparable s
#If we don't have comparable scales, then some of the coefficients as obtained by fitti
#This might become very annoying at the time of model evaluation.  
##So it is advised to use standardization or normalization so that the units of the coef
 
#As you know, there are two common ways of rescaling:
#1. Min-Max scaling 
#2. Standardisation (mean-0, sigma-1) 
 
import warnings 
warnings.filterwarnings('ignore') 
 
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
 
# define standard scaler 
#scaler = StandardScaler() 
scaler = MinMaxScaler() 
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])
df_Newtrain.head(20) 

In [ ]:
y_Newtrain = df_Newtrain.pop('G3') 
x_Newtrain = df_Newtrain
y_Newtest = df_Newtest.pop('G3')
x_Newtest = df_Newtest

In [ ]:
y_Newtrain = torch.tensor(y_Newtrain.values).float()
y_Newtrain = y_Newtrain[:,None]
x_Newtrain = torch.tensor(x_Newtrain.values).float()
y_Newtest = torch.tensor(y_Newtest.values).float()
y_Newtest = y_Newtest[:,None]
x_Newtest = torch.tensor(x_Newtest.values).float()

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
seq_model1 = nn.Sequential(
    nn.Linear(11,316),
    nn.Tanh(),
    nn.Linear(316, 11))
seq_model1

In [ ]:
#seq_model2 = nn.Sequential(
#    nn.Linear(5,8),
 #   nn.Tanh(),
  #  nn.Linear(8,4),
   # nn.Tanh(),
    #nn.Linear(4,2),
    #nn.Tanh(),
    #nn.Linear(2, 5))
#seq_model2

In [ ]:
optimizer = optim.SGD(seq_model1.parameters(), lr=1e-3)
training_loop(200, optimizer, seq_model1, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)

In [ ]:
#optimizer = optim.SGD(seq_model2.parameters(), lr=1e-3)
#training_loop(200, optimizer, seq_model2, nn.MSELoss(), x_Newtrain, x_Newtest, y_Newtrain, y_Newtest)